In [32]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import sys
import matplotlib.pyplot as plt

In [33]:
def delUnname0(df):
    df = df.drop('Unnamed: 0',axis=1)
    return df
def proper_model(data_ts, maxLag):
    init_bic = sys.maxint
    init_p = 0
    init_q = 0
    init_properModel = None
    for p in np.arange(maxLag):
        for q in np.arange(maxLag):
            model = sm.tsa.ARMA(data_ts, order=(p, q),freq='D')
            try:
                results_ARMA = model.fit(disp=-1, method='css')
            except:
                continue
            bic = results_ARMA.bic
            if bic < init_bic:
                init_p = p
                init_q = q
                init_properModel = results_ARMA
                init_bic = bic
    return init_bic, init_p, init_q, init_properModel

In [35]:
Customer_Flow = pd.read_csv("Customer_Flow.csv")
Customer_Flow = delUnname0(Customer_Flow)
Customer_Flow['data'] = pd.to_datetime(Customer_Flow['data'])
# 第1629,1683 出现一点小问题,先存上
p_result = []
errorshop=[1629,1683]
for shop_id,eachShop in Customer_Flow.groupby(['shop_id']):
    print shop_id
    if shop_id > 1683:
        try:
            eachShop.index = eachShop['data']
            eachShop = eachShop.drop(['data','shop_id'],axis=1)
            #这里要处理一个问题就是假期问题,这里拿到的数据是不完全数据,即是有一些商店有一些时间段根本没有数据这里需要注意
            eachShop_series = pd.Series(index=eachShop.index,data=eachShop.Num)
            # 数据预处理 不一定是取log,可能差分呢,也可能小波,也有可能卡尔曼傅里叶,也可以把序列进行分解然后进行拟合
            ts_log = np.log(eachShop_series)
            #  这里使用模型参数自动识别
            init_bic, init_p, init_q, init_properModel = proper_model(ts_log,9)
            print 'shop_id',shop_id,'bic:', init_bic, 'p:', init_p, 'q:',init_q
            # 预测结果还原
            # There is no frequency for these dates and date 2016-11-01 00:00:00 is not in dates index. Try giving a date that is in the dates index or use an integer
            # If the model is an ARMAX and out-of-sample forecasting is requested, exog must be given. Note that you’ll need to pass k_ar additional lags for any exogenous variables.
            #exog = observations of (p + 14)
            # predict_ts = init_properModel.predict(start=p_startt,end=p_endt,exog=observation)
            predict_ts = init_properModel.predict(start="2016-10-30",end="2016-11-14")
            log_recover = np.exp(predict_ts)
            log_recover.dropna(inplace=True)
            result = pd.DataFrame(log_recover)  
            result = result.rename(columns={0:'Num'})
            each_line = []
            each_line.append(shop_id)
            for Num in result.Num:
                each_line.append(Num) 
            p_result.append(each_line)
        except ValueError, e:
            errorshop.append(shop_id)
            print 'ValueError:note down error shop id~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~' 
            continue  
    else:
        continue;

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0
185.

/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python

shop_id 1684.0 bic: 574.462964495 p: 8 q: 4
1685.0


/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python

shop_id 1685.0 bic: 231.492024748 p: 3 q: 1
1686.0


...

/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python

shop_id 1999.0 bic: -86.9665016272 p: 7 q: 2
2000.0


/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/lib/python

shop_id 2000.0 bic: 215.535770143 p: 7 q: 4


/usr/lib/python2.7/dist-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [12]:
# 预测结果作图 抽取oneshop to do predict
# ts_plt = shop_1_series[log_recover.index]
# plt.figure(facecolor='white')
# log_recover.plot(color='blue', label='Predict',figsize=(24, 16))
# ts_plt.plot(color='red', label='Original')
# plt.legend(loc='best')
# plt.title('RMSE: %.4f'% np.sqrt(sum((log_recover-ts_plt)**2)/ts_plt.size))
# plt.show()

In [38]:
errorshop

[1690.0, 1707.0, 1716.0, 1824.0, 1831.0, 1862.0]

In [ ]:
p_result.remove(1629)

In [44]:
p_result.remove(1683)

In [45]:
p_result

[[1684.0,
  90.980025732957429,
  84.177696339966189,
  68.252605502116467,
  71.405014664337912,
  73.474224533768449,
  79.427851016685324,
  82.466605833138075,
  77.472292754721309,
  66.384661719312888,
  59.780957369550919,
  58.611716481682613,
  64.094096895608502,
  70.216680569591333,
  73.051030649278829,
  67.419433102557491,
  59.5814644959963],
 [1685.0,
  85.486750454355885,
  87.441145026347954,
  94.807697277970433,
  90.772339675642428,
  88.651380839014493,
  88.262757447225141,
  88.322977852234501,
  88.16904112936021,
  87.784634192139549,
  87.341518663370991,
  86.936039489559448,
  86.573405051919508,
  86.230261108535359,
  85.892432467062719,
  85.558407963574652,
  85.231029358390089],
 [1686.0,
  26.466413351248512,
  54.158469106322649,
  67.621142221821373,
  54.433199184033136,
  38.880394309734577,
  45.499996189257431,
  21.879067770981909,
  23.985427237497834,
  66.120182472561197,
  61.149262010192054,
  50.71143869552202,
  40.379416582343659,
  42

In [48]:
p_result = abs(pd.DataFrame(p_result).astype(int)) 
p_result.to_csv('ARMA1684_end.csv',header=False,index=False,encoding='utf-8')